In [272]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error

In [273]:
in_file_name = 'data/grouped_paces_ju.tsv'
df_all = pd.read_csv(in_file_name, delimiter="\t")
history = df_all

In [274]:
order18 = pd.read_csv('data/running_order_j2018_ju.tsv', delimiter="\t")


In [275]:
#paces = df.as_matrix(["pace_1", "pace_2", "pace_3", "pace_4", "pace_5", "pace_6"])
paces = history[["pace_1", "pace_2", "pace_3", "pace_4", "pace_5", "pace_6"]]
logs = np.log(paces)
means = np.nanmean(logs, axis=1)
stdevs = np.nanstd(logs, axis=1)
history = history.assign(log_means=pd.Series(means).values)
history = history.assign(log_stdevs=pd.Series(stdevs).values)

/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice
  after removing the cwd from sys.path.
/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1434: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [276]:
# Estimate values for all, but only use them if not other value is available
with_history = history[np.isfinite(df_all.pace_2)]
x = with_history.mean_team_id.values
x = x.reshape(len(x), 1)

log_means = with_history.log_means.values.reshape(len(with_history.log_means), 1)
log_means_model = linear_model.LinearRegression()
log_means_model.fit(x, log_means)

estimated_log_means = log_means_model.predict(order18.team_id.values.reshape(len(order18.team_id), 1))
order18 = order18.assign(estimated_log_means=estimated_log_means)

log_stdevs = with_history.log_stdevs.values.reshape(len(with_history.log_stdevs), 1)
log_stdevs_model = linear_model.LinearRegression()
log_stdevs_model.fit(x, log_stdevs)

estimated_log_stdevs = log_stdevs_model.predict(order18.team_id.values.reshape(len(order18.team_id), 1))
order18 = order18.assign(estimated_log_stdevs=estimated_log_stdevs)
order18.head()

,team_id,team,team_base_name,leg,leg_dist,name,estimated_log_means,estimated_log_stdevs
0,4,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,1,11.0,-,1.944439,0.084991
1,4,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,2,11.9,-,1.944439,0.084991
2,4,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,3,12.8,-,1.944439,0.084991
3,4,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,4,8.7,-,1.944439,0.084991
4,4,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,5,8.7,-,1.944439,0.084991


In [277]:
# Combine history with 2018 runners 
no_history_row = pd.DataFrame([[0,0]], columns=["log_means", "log_stdevs"])
def get_history_row(running_order_row):
    name = running_order_row["name"].lower()
    
    by_name = history[history['name'] == name]
    by_name_and_colon = history[history['name'].str.contains(name + ":", regex=False)]

    runners = by_name.append(by_name_and_colon)
    if(len(runners) == 1):
        return runners
    team_name = running_order_row["team_base_name"].upper()
    runners = runners[runners['teams'].str.contains(team_name, regex=False)]
    if(len(runners) == 1):
        return runners
    if(len(runners) == 0):
        return no_history_row
    print(f"name '{name}' team_name '{team_name}'")
    print(f"by_name {len(by_name)} by_name_and_colon {len(by_name_and_colon)} runners {len(runners)}")
    print(f"Duplicate runner {runners}")
    #print(f"TEAMS by_name_and_colon {by_name_and_colon['teams']}")
    return runners.sort_values("num_runs", ascending = False).head(1)

def get_estimate_params(running_order_row):
    history_row = get_history_row(running_order_row)
    #print(f"estimate_row log_means {history_row.log_means} {history_row.log_stdevs}")
    log_means = history_row.log_means.values[0]
    log_stdevs = history_row.log_stdevs.values[0]
    return pd.Series({"history_log_means": log_means, "history_log_stdevs": log_stdevs})

order18 = order18[order18['team'].str.contains("Reak") | order18['team'].str.contains("Puskasil") | order18['team'].str.contains("aukat")]
estimate_params = order18.apply(lambda row: get_estimate_params(row), axis=1)
order18 = order18.assign(history_log_means = estimate_params.history_log_means)
order18 = order18.assign(history_log_stdevs = estimate_params.history_log_stdevs)

In [278]:
order18['log_means'] = np.where(np.isfinite(order18["history_log_means"]) & order18["history_log_means"] > 0, order18["history_log_means"], order18["estimated_log_means"])
order18['log_stdevs'] = np.where(np.isfinite(order18["history_log_stdevs"]) & order18["history_log_stdevs"] > 0, order18["history_log_stdevs"], order18["estimated_log_stdevs"])
order18.head()

,team_id,team,team_base_name,leg,leg_dist,name,estimated_log_means,estimated_log_stdevs,history_log_means,history_log_stdevs,log_means,log_stdevs
928,429,Reaktor Innovations 1,Reaktor Innovations,1,11.0,Mikko Peltonen,2.128667,0.092121,2.184951,0.081299,2.184951,0.081299
929,429,Reaktor Innovations 1,Reaktor Innovations,2,11.9,Pasi Huhtiniemi,2.128667,0.092121,2.222239,0.072482,2.222239,0.072482
930,429,Reaktor Innovations 1,Reaktor Innovations,3,12.8,Joni Freeman,2.128667,0.092121,2.130879,0.113846,2.130879,0.113846
931,429,Reaktor Innovations 1,Reaktor Innovations,4,8.7,Tuomas Kareinen,2.128667,0.092121,2.293240,0.160070,2.293240,0.160070
932,429,Reaktor Innovations 1,Reaktor Innovations,5,8.7,Janne Vaittinen,2.128667,0.092121,2.324384,0.039861,2.324384,0.039861


In [279]:
# Calculate personal estimates
# s = sigma and scale = exp(mu).

log_means = np.exp(order18['log_means']) 
log_stdevs = order18['log_stdevs']

intervals95 = lognorm.interval(0.95, s = log_stdevs, scale = log_means)
means = lognorm.mean(s = log_stdevs, scale = log_means)
medians = lognorm.median(s = log_stdevs, scale = log_means)

In [280]:
order18 = order18.assign(ind_95_start = pd.Series(intervals95[0] * order18.leg_dist).values)
order18 = order18.assign(ind_95_end = pd.Series(intervals95[1] * order18.leg_dist).values)
order18 = order18.assign(ind_mean = pd.Series(means * order18.leg_dist).values)
order18 = order18.assign(ind_median = pd.Series(medians * order18.leg_dist).values)

In [281]:
order18.head()

,team_id,team,team_base_name,leg,leg_dist,name,estimated_log_means,estimated_log_stdevs,history_log_means,history_log_stdevs,log_means,log_stdevs,ind_95_start,ind_95_end,ind_mean,ind_median
928,429,Reaktor Innovations 1,Reaktor Innovations,1,11.0,Mikko Peltonen,2.128667,0.092121,2.184951,0.081299,2.184951,0.081299,83.388003,114.684964,98.116094,97.792383
929,429,Reaktor Innovations 1,Reaktor Innovations,2,11.9,Pasi Huhtiniemi,2.128667,0.092121,2.222239,0.072482,2.222239,0.072482,95.270067,126.575551,110.101683,109.812846
930,429,Reaktor Innovations 1,Reaktor Innovations,3,12.8,Joni Freeman,2.128667,0.092121,2.130879,0.113846,2.130879,0.113846,86.244880,134.754922,108.505909,107.805019
931,429,Reaktor Innovations 1,Reaktor Innovations,4,8.7,Tuomas Kareinen,2.128667,0.092121,2.293240,0.160070,2.293240,0.160070,62.980977,117.953879,87.302084,86.190780
932,429,Reaktor Innovations 1,Reaktor Innovations,5,8.7,Janne Vaittinen,2.128667,0.092121,2.324384,0.039861,2.324384,0.039861,82.235006,96.142589,88.987969,88.917301


In [282]:
by_teams = order18.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                                \
leg                          1                      2                      3   
team_id                                                                        
429      Reaktor Innovations 1  Reaktor Innovations 1  Reaktor Innovations 1   
1131         Puskasilimät OK 1      Puskasilimät OK 1      Puskasilimät OK 1   
1270     Reaktor Innovations 2  Reaktor Innovations 2  Reaktor Innovations 2   
1541             Tuurihaukat 1          Tuurihaukat 1          Tuurihaukat 1   

                                                                              \
leg                          4                      5                      6   
team_id                                                                        
429      Reaktor Innovations 1  Reaktor Innovations 1  Reaktor Innovations 1   
1131         Puskasilimät OK 1      Puskasilimät OK 1      Puskasilimät OK 1   
1270     Reaktor Innovations 2  Reaktor Innovations 2  Reaktor Innovations 2   
1541             Tuurihaukat 1          Tuurihaukat 1          Tuurihaukat 1   

                                     team_base_name                       \
leg                          7                    1                    2   
team_id                                                                    
429      Reaktor Innovations 1  Reaktor Innovations  Reaktor Innovations   
1131         Puskasilimät OK 1      Puskasilimät OK      Puskasilimät OK   
1270     Reaktor Innovations 2  Reaktor Innovations  Reaktor Innovations   
1541             Tuurihaukat 1          Tuurihaukat          Tuurihaukat   

                                 ...        ind_mean                          \
leg                        3     ...               5           6           7   
team_id                          ...                                           
429      Reaktor Innovations     ...       88.987969   91.145398  161.474599   
1131         Puskasilimät OK     ...       99.652408  123.706438  176.548436   
1270     Reaktor Innovations     ...       98.210347  131.421494  157.389252   
1541             Tuurihaukat     ...      100.148786  112.533154  161.350836   

         ind_median                                                 \
leg               1           2           3           4          5   
team_id                                                              
429       97.792383  109.812846  107.805019   86.190780  88.917301   
1131     124.333000  131.055553  146.752693  110.577000  99.115993   
1270     116.864000  133.115223  178.927107   67.174301  96.512277   
1541     104.550011  125.580521  110.462555   99.845969  99.248094   

                                 
leg               6           7  
team_id                          
429       90.759473  161.284847  
1131     123.040544  175.598100  
1270     130.682049  156.503700  
1541     111.844800  160.540688  

[4 rows x 98 columns]

In [283]:
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1,8):
        samples[i] = row["leg_dist"][i] * lognorm.rvs(s = row["log_stdevs"][i], scale = np.exp(row["log_means"][i]), size = 10000)

    samples_sums = pd.DataFrame()
    for i in range(1,8):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    """
    for i in range(1,8):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1,8):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1,8)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1,8)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1,8)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1,8)}
    new_cols = {**fin_start95_dict, **fin_end95_dict, **fin_median_dict, **fin_mean_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_start95_5,fin_start95_6,fin_start95_7,fin_end95_1,fin_end95_2,fin_end95_3,...,fin_median_5,fin_median_6,fin_median_7,fin_mean_1,fin_mean_2,fin_mean_3,fin_mean_4,fin_mean_5,fin_mean_6,fin_mean_7
team_id,,,,,,,,,,,,,,,,,,,,,
429,83.183290,186.726524,285.400004,362.701578,451.016852,539.408365,699.145274,115.141318,231.817613,350.509796,...,492.573897,583.513254,744.943972,98.142317,208.272472,316.583806,404.051601,493.060972,584.152300,745.678726
1131,101.858807,230.941743,374.975023,478.151447,573.682167,690.166627,853.912685,152.098092,284.716702,434.163316,...,613.779973,737.113370,914.019862,125.173426,256.298170,403.214538,514.568834,614.190627,737.780402,914.308446
1270,94.815494,220.211136,365.279204,433.156854,521.724494,647.296801,799.903008,143.516210,284.679599,519.795254,...,597.533329,728.970361,886.137003,117.337387,250.870569,433.864656,501.255202,599.939301,731.462652,888.780361
1541,81.407335,200.348903,309.776106,400.738147,493.922231,600.555535,754.032558,134.019224,264.826171,376.855903,...,541.886009,654.301484,816.171786,105.334327,231.143206,341.698168,442.592574,542.796513,655.333465,816.832455


In [284]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()
by_teams_flat

,team_1,team_2,team_3,team_4,team_5,team_6,team_7,team_base_name_1,team_base_name_2,team_base_name_3,...,ind_mean_5,ind_mean_6,ind_mean_7,ind_median_1,ind_median_2,ind_median_3,ind_median_4,ind_median_5,ind_median_6,ind_median_7
team_id,,,,,,,,,,,,,,,,,,,,,
429,Reaktor Innovations 1,Reaktor Innovations 1,Reaktor Innovations 1,Reaktor Innovations 1,Reaktor Innovations 1,Reaktor Innovations 1,Reaktor Innovations 1,Reaktor Innovations,Reaktor Innovations,Reaktor Innovations,...,88.987969,91.145398,161.474599,97.792383,109.812846,107.805019,86.190780,88.917301,90.759473,161.284847
1131,Puskasilimät OK 1,Puskasilimät OK 1,Puskasilimät OK 1,Puskasilimät OK 1,Puskasilimät OK 1,Puskasilimät OK 1,Puskasilimät OK 1,Puskasilimät OK,Puskasilimät OK,Puskasilimät OK,...,99.652408,123.706438,176.548436,124.333000,131.055553,146.752693,110.577000,99.115993,123.040544,175.598100
1270,Reaktor Innovations 2,Reaktor Innovations 2,Reaktor Innovations 2,Reaktor Innovations 2,Reaktor Innovations 2,Reaktor Innovations 2,Reaktor Innovations 2,Reaktor Innovations,Reaktor Innovations,Reaktor Innovations,...,98.210347,131.421494,157.389252,116.864000,133.115223,178.927107,67.174301,96.512277,130.682049,156.503700
1541,Tuurihaukat 1,Tuurihaukat 1,Tuurihaukat 1,Tuurihaukat 1,Tuurihaukat 1,Tuurihaukat 1,Tuurihaukat 1,Tuurihaukat,Tuurihaukat,Tuurihaukat,...,100.148786,112.533154,161.350836,104.550011,125.580521,110.462555,99.845969,99.248094,111.844800,160.540688


In [285]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [286]:
# Convert minutes to date and times
start_timestamp = pd.Timestamp(year = 2018, month = 6, day = 16, hour = 23)

for leg in range(1,8):
    estimates[f"fint_median_{leg}"] = pd.to_datetime(estimates[f"fin_median_{leg}"] * 60, unit = "s", origin= start_timestamp)
    estimates[f"fint_start95_{leg}"] = pd.to_datetime(estimates[f"fin_start95_{leg}"] * 60, unit = "s", origin= start_timestamp)
    estimates[f"fint_end95_{leg}"] = pd.to_datetime(estimates[f"fin_end95_{leg}"] * 60, unit = "s", origin= start_timestamp)

estimates["fint_median_1"]  

team_id
429    2018-06-17 00:37:50.751292
1131   2018-06-17 01:04:28.923140
1270   2018-06-17 00:56:44.730707
1541   2018-06-17 00:44:39.278856
Name: fint_median_1, dtype: datetime64[ns]

In [287]:
estimates.to_csv('data/team_estimates_ju2018.tsv', sep="\t")

In [288]:

estimates[[f"{field_prefix}_{leg}" for leg in range(1,8) for field_prefix in ["name", "fint_median", "ind_median"]]]    

,name_1,fint_median_1,ind_median_1,name_2,fint_median_2,ind_median_2,name_3,fint_median_3,ind_median_3,name_4,...,ind_median_4,name_5,fint_median_5,ind_median_5,name_6,fint_median_6,ind_median_6,name_7,fint_median_7,ind_median_7
team_id,,,,,,,,,,,,,,,,,,,,,
429,Mikko Peltonen,2018-06-17 00:37:50.751292,97.792383,Pasi Huhtiniemi,2018-06-17 02:27:53.958676,109.812846,Joni Freeman,2018-06-17 04:15:57.105165,107.805019,Tuomas Kareinen,...,86.190780,Janne Vaittinen,2018-06-17 07:12:34.433795,88.917301,Olavi Kanerva,2018-06-17 08:43:30.795218,90.759473,Jyri Kytömäki,2018-06-17 11:24:56.638310,161.284847
1131,Tuomas Ketonen,2018-06-17 01:04:28.923140,124.333000,Petri Miettinen,2018-06-17 03:15:51.488206,131.055553,Jaakko Havola,2018-06-17 05:42:42.652798,146.752693,Joonas Kekäle,...,110.577000,Joonas Mikkonen,2018-06-17 09:13:46.798351,99.115993,Esko Harjama,2018-06-17 11:17:06.802203,123.040544,Anssi Pesonen,2018-06-17 14:14:01.191734,175.598100
1270,Antti-Ville Jokela,2018-06-17 00:56:44.730707,116.864000,Karri-Pekka Laakso,2018-06-17 03:10:23.347749,133.115223,Oskari Pirttikoski,2018-06-17 06:11:32.675529,178.927107,Jeppe Koivula,...,67.174301,Mikko Apo,2018-06-17 08:57:31.999739,96.512277,Tommi Simell,2018-06-17 11:08:58.221660,130.682049,Aki Kaivola,2018-06-17 13:46:08.220164,156.503700
1541,Niklas Pigg,2018-06-17 00:44:39.278856,104.550011,Aleksi Heikkilä,2018-06-17 02:50:27.370332,125.580521,Arttu Heikkilä,2018-06-17 04:41:02.825597,110.462555,Vesa Riihimäki,...,99.845969,Pasi Riihimäki,2018-06-17 08:01:53.160530,99.248094,Eero Heikkilä,2018-06-17 09:54:18.089050,111.844800,Perttu Heikkilä,2018-06-17 12:36:10.307148,160.540688


In [289]:
first_names = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], first_names))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_").sort_values(by=['team_id', 'leg'])
runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
#column_base_names
runner_estimates.head()

team  leg_dist             name  \
team_id leg                                                     
429     1    Reaktor Innovations 1      11.0   Mikko Peltonen   
        2    Reaktor Innovations 1      11.9  Pasi Huhtiniemi   
        3    Reaktor Innovations 1      12.8     Joni Freeman   
        4    Reaktor Innovations 1       8.7  Tuomas Kareinen   
        5    Reaktor Innovations 1       8.7  Janne Vaittinen   

             history_log_means  history_log_stdevs  log_means  log_stdevs  \
team_id leg                                                                 
429     1             2.184951            0.081299   2.184951    0.081299   
        2             2.222239            0.072482   2.222239    0.072482   
        3             2.130879            0.113846   2.130879    0.113846   
        4             2.293240            0.160070   2.293240    0.160070   
        5             2.324384            0.039861   2.324384    0.039861   

             ind_95_start  ind_95_end    ind_mean  ind_median  fin_start95  \
team_id leg                                                                  
429     1       83.388003  114.684964   98.116094   97.792383    83.183290   
        2       95.270067  126.575551  110.101683  109.812846   186.726524   
        3       86.244880  134.754922  108.505909  107.805019   285.400004   
        4       62.980977  117.953879   87.302084   86.190780   362.701578   
        5       82.235006   96.142589   88.987969   88.917301   451.016852   

              fin_end95  fin_median    fin_mean                fint_median  \
team_id leg                                                                  
429     1    115.141318   97.845855   98.142317 2018-06-17 00:37:50.751292   
        2    231.817613  207.899311  208.272472 2018-06-17 02:27:53.958676   
        3    350.509796  315.951753  316.583806 2018-06-17 04:15:57.105165   
        4    449.694580  403.414993  404.051601 2018-06-17 05:43:24.899598   
        5    539.135812  492.573897  493.060972 2018-06-17 07:12:34.433795   

                          fint_start95                 fint_end95  
team_id leg                                                        
429     1   2018-06-17 00:23:10.997427 2018-06-17 00:55:08.479071  
        2   2018-06-17 02:06:43.591428 2018-06-17 02:51:49.056769  
        3   2018-06-17 03:45:24.000250 2018-06-17 04:50:30.587761  
        4   2018-06-17 05:02:42.094688 2018-06-17 06:29:41.674801  
        5   2018-06-17 06:31:01.011140 2018-06-17 07:59:08.148703

In [290]:
runner_estimates.to_csv('data/runner_estimates_ju2018.tsv', sep="\t")

In [291]:
column_base_names

['team',
 'team_base_name',
 'leg_dist',
 'name',
 'estimated_log_means',
 'estimated_log_stdevs',
 'history_log_means',
 'history_log_stdevs',
 'log_means',
 'log_stdevs',
 'ind_95_start',
 'ind_95_end',
 'ind_mean',
 'ind_median',
 'fin_start95',
 'fin_end95',
 'fin_median',
 'fin_mean',
 'fint_median',
 'fint_start95',
 'fint_end95']

In [292]:
runner_estimates[['team',
 'name',
 'leg_dist',
 'ind_95_start',
 'ind_95_end',
 'ind_median',
 'fin_median',
 'fint_median',
 'fint_start95',
 'fint_end95']]

team                name  leg_dist  \
team_id leg                                                        
429     1    Reaktor Innovations 1      Mikko Peltonen      11.0   
        2    Reaktor Innovations 1     Pasi Huhtiniemi      11.9   
        3    Reaktor Innovations 1        Joni Freeman      12.8   
        4    Reaktor Innovations 1     Tuomas Kareinen       8.7   
        5    Reaktor Innovations 1     Janne Vaittinen       8.7   
        6    Reaktor Innovations 1       Olavi Kanerva      10.8   
        7    Reaktor Innovations 1       Jyri Kytömäki      15.3   
1131    1        Puskasilimät OK 1      Tuomas Ketonen      11.0   
        2        Puskasilimät OK 1     Petri Miettinen      11.9   
        3        Puskasilimät OK 1       Jaakko Havola      12.8   
        4        Puskasilimät OK 1       Joonas Kekäle       8.7   
        5        Puskasilimät OK 1     Joonas Mikkonen       8.7   
        6        Puskasilimät OK 1        Esko Harjama      10.8   
        7        Puskasilimät OK 1       Anssi Pesonen      15.3   
1270    1    Reaktor Innovations 2  Antti-Ville Jokela      11.0   
        2    Reaktor Innovations 2  Karri-Pekka Laakso      11.9   
        3    Reaktor Innovations 2  Oskari Pirttikoski      12.8   
        4    Reaktor Innovations 2       Jeppe Koivula       8.7   
        5    Reaktor Innovations 2           Mikko Apo       8.7   
        6    Reaktor Innovations 2        Tommi Simell      10.8   
        7    Reaktor Innovations 2         Aki Kaivola      15.3   
1541    1            Tuurihaukat 1         Niklas Pigg      11.0   
        2            Tuurihaukat 1     Aleksi Heikkilä      11.9   
        3            Tuurihaukat 1      Arttu Heikkilä      12.8   
        4            Tuurihaukat 1      Vesa Riihimäki       8.7   
        5            Tuurihaukat 1      Pasi Riihimäki       8.7   
        6            Tuurihaukat 1       Eero Heikkilä      10.8   
        7            Tuurihaukat 1     Perttu Heikkilä      15.3   

             ind_95_start  ind_95_end  ind_median  fin_median  \
team_id leg                                                     
429     1       83.388003  114.684964   97.792383   97.845855   
        2       95.270067  126.575551  109.812846  207.899311   
        3       86.244880  134.754922  107.805019  315.951753   
        4       62.980977  117.953879   86.190780  403.414993   
        5       82.235006   96.142589   88.917301  492.573897   
        6       75.766684  108.719051   90.759473  583.513254   
        7      146.661391  177.366392  161.284847  744.943972   
1131    1      101.425756  152.413899  124.333000  124.482052   
        2      122.124961  140.639210  131.055553  255.858137   
        3      135.819675  158.565783  146.752693  402.710880   
        4       90.204176  135.551074  110.577000  514.031878   
        5       80.854757  121.501572   99.115993  613.779973   
        6      100.371423  150.829538  123.040544  737.113370   
        7      143.245719  215.257341  175.598100  914.019862   
1270    1       94.898137  143.914254  116.864000  116.745512   
        2      113.387296  156.275556  133.115223  250.389129   
        3      122.315607  261.740185  178.927107  431.544592   
        4       58.132571   77.622350   67.174301  498.901060   
        5       66.927468  139.174840   96.512277  597.533329   
        6      106.118933  160.930737  130.682049  728.970361   
        7      127.087123  192.729268  156.503700  886.137003   
1541    1       81.293071  134.460472  104.550011  104.654648   
        2      108.673765  145.117519  125.580521  230.456172   
        3      102.368656  119.196408  110.462555  341.047093   
        4       74.723536  133.414693   99.845969  441.420738   
        5       76.261369  129.163485   99.248094  541.886009   
        6       90.016659  138.966048  111.844800  654.301484   
        7      131.879938  195.430122  160.540688  816.171786   

                           fint_median